In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
DATA_VERSION = "v003"
TRIAL_NO = "059"
seed = 2069

In [15]:
path = f"../log/{DATA_VERSION}_{TRIAL_NO}/importance_fc_{DATA_VERSION}_{TRIAL_NO}_{seed}.csv"
importance_df = pd.read_csv(path, index_col=0)
importance_df = importance_df.pivot_table(values="importance", columns="fold", index="feature")
importance_df["ave"] = importance_df.mean(axis=1)
importance_df.sort_values("ave", inplace=True, ascending=False)
importance_df = importance_df.reset_index()

In [17]:
yiemon_HnJ = [
'angle_C_from2nd_0',
 'angle_C_from2nd_0_ratio',
 'angle_C_from2nd_1',
 'angle_C_from2nd_1_ratio',
 'angle_C_from2nd_2',
 'angle_C_from2nd_2_ratio',
 'angle_H_from2nd_0',
 'angle_H_from2nd_0_ratio',
 'angle_H_from2nd_1',
 'angle_H_from2nd_1_ratio',
 'angle_N_from2nd_0',
 'angle_N_from2nd_0_ratio',
 'angle_N_from2nd_1',
 'angle_N_from2nd_1_ratio',
 'angle_O_from2nd_0',
 'angle_O_from2nd_0_ratio',
 'angle_O_from2nd_1',
 'angle_O_from2nd_1_ratio',
 'count_dist_C_from1st',
 'count_dist_C_from2nd',
 'count_dist_H_from1st',
 'count_dist_H_from2nd',
 'count_dist_N_from1st',
 'count_dist_N_from2nd',
 'count_dist_O_from1st',
 'count_dist_O_from2nd',
 'd_C_from1st_0',
 'd_C_from1st_1',
 'd_C_from1st_2',
 'd_C_from1st_ratio_0',
 'd_C_from1st_ratio_1',
 'd_C_from1st_ratio_2',
 'd_C_from2nd_0',
 'd_C_from2nd_1',
 'd_C_from2nd_2',
 'd_C_from2nd_ratio_0',
 'd_C_from2nd_ratio_1',
 'd_C_from2nd_ratio_2',
 'd_H_from1st_0',
 'd_H_from1st_1',
 'd_H_from1st_ratio_0',
 'd_H_from1st_ratio_1',
 'd_H_from2nd_0',
 'd_H_from2nd_1',
 'd_H_from2nd_ratio_0',
 'd_H_from2nd_ratio_1',
 'd_N_from1st_0',
 'd_N_from1st_1',
 'd_N_from1st_ratio_0',
 'd_N_from1st_ratio_1',
 'd_N_from2nd_0',
 'd_N_from2nd_1',
 'd_N_from2nd_ratio_0',
 'd_N_from2nd_ratio_1',
 'd_O_from1st_0',
 'd_O_from1st_1',
 'd_O_from1st_ratio_0',
 'd_O_from1st_ratio_1',
 'd_O_from2nd_0',
 'd_O_from2nd_1',
 'd_O_from2nd_ratio_0',
 'd_O_from2nd_ratio_1',
 'eem_length2',
 'eem_x',
 'gasteiger_length2',
 'gasteiger_x',
 'mean_angle_C_from2nd',
 'mean_angle_H_from2nd',
 'mean_angle_N_from2nd',
 'mean_angle_O_from2nd',
 'mean_dist_C_from1st',
 'mean_dist_C_from2nd',
 'mean_dist_H_from1st',
 'mean_dist_H_from2nd',
 'mean_dist_N_from1st',
 'mean_dist_N_from2nd',
 'mean_dist_O_from1st',
 'mean_dist_O_from2nd',
 'mean_dist_ratio_C_from1st',
 'mean_dist_ratio_C_from2nd',
 'mean_dist_ratio_H_from1st',
 'mean_dist_ratio_H_from2nd',
 'mean_dist_ratio_N_from1st',
 'mean_dist_ratio_N_from2nd',
 'mean_dist_ratio_O_from1st',
 'mean_dist_ratio_O_from2nd',
 'mmff94_length2',
 'mmff94_x',
 'qeq_length2',
 'qeq_x',
 'std_angle_C_from2nd',
 'std_angle_H_from2nd',
 'std_angle_N_from2nd',
 'std_angle_O_from2nd',
 'std_dist_C_from1st',
 'std_dist_C_from2nd',
 'std_dist_H_from1st',
 'std_dist_H_from2nd',
 'std_dist_N_from1st',
 'std_dist_N_from2nd',
 'std_dist_O_from1st',
 'std_dist_O_from2nd']

In [19]:
importance_df["ratio"] = importance_df["ave"]/importance_df["ave"].sum()

In [20]:
importance_df

fold,feature,1,2,3,4,5,ave,ratio
0,1J1st_AverageBondAngle,"10,115,884,918.28777","10,104,623,380.19084","10,077,264,638.47040","10,084,160,112.46832","10,080,560,787.03906","10,092,498,767.29128",0.91468
1,dist_C_0_x,"280,120,014.54616","279,833,531.23166","279,593,835.31282","279,821,082.72638","279,872,226.20915","279,848,138.00524",0.02536
2,molecule_type_dist_std_diff,"115,105,961.16502","114,715,012.15516","114,084,182.62603","114,797,520.95381","114,808,629.88491","114,702,261.35698",0.01040
3,eem2015ha_1,"102,088,081.00437","100,064,969.96234","100,210,704.75236","66,487,691.59089","99,173,690.76682","93,605,027.61536",0.00848
4,dist_to_type_mean,"81,244,121.25743","89,462,339.00229","77,507,890.13287","90,773,896.35203","90,054,689.16886","85,808,587.18270",0.00778
5,1Jlast_GetPartialCharge,"46,833,816.37383","47,060,510.64718","74,688,681.82621","67,855,604.92062","61,312,015.03210","59,550,125.75999",0.00540
6,mmff94_0,"45,796,584.16040","46,252,482.76216","46,347,976.04719","46,407,730.69059","43,156,997.36417","45,592,354.20490",0.00413
7,2J2nd_AverageBondAngle,"41,803,603.32371","41,356,164.24348","40,960,581.55232","41,677,370.80667","42,094,992.12510","41,578,542.41026",0.00377
8,1J1st_AveSmallestBondAngle_diff,"23,456,526.97180","33,096,529.84937","37,730,051.26893","25,035,991.40652","23,994,268.74588","28,662,673.64850",0.00260
9,cosT,"22,968,294.08123","22,759,577.35705","22,757,029.44653","22,737,032.08236","22,601,136.37542","22,764,613.86852",0.00206


In [22]:
importance_df_y = importance_df[importance_df.feature.isin(yiemon_HnJ)]

In [26]:
importance_df_y_high = importance_df_y[importance_df_y.ratio >= 0.00005]
importance_df_y_high

fold,feature,1,2,3,4,5,ave,ratio
13,mean_dist_C_from2nd,"2,219,107.80720","1,082,870.41410","12,345,291.11046","5,667,477.47080","12,604,486.16454","6,783,846.59342",0.00061
17,mean_angle_C_from2nd,"4,464,357.23324","4,365,691.19720","5,100,971.37583","4,677,631.60805","6,538,590.93950","5,029,448.47077",0.00046
36,d_O_from2nd_ratio_0,"1,745,468.21043","1,250,039.08453","1,385,857.75820","1,378,011.21881","1,703,789.89275","1,492,633.23294",0.00014
39,mean_dist_ratio_C_from2nd,"1,174,243.71498","1,314,657.43831","1,302,007.18286","1,324,328.27011","1,197,997.00485","1,262,646.72222",0.00011
40,mean_dist_ratio_O_from1st,"2,685,933.43542","241,324.03499","236,974.91415","2,122,986.04824","918,808.59357","1,241,205.40528",0.00011
41,mean_dist_ratio_O_from2nd,"548,789.52500","2,173,530.02903","1,247,912.36466","954,880.55809","1,167,358.27827","1,218,494.15101",0.00011
42,d_O_from1st_ratio_0,"227,104.04968","1,299,200.63347","2,450,670.81099","631,376.41198","1,346,113.18302","1,190,893.01783",0.00011
56,mean_angle_O_from2nd,"505,996.26947","495,110.14479","1,123,202.96260","889,573.24679","806,021.17185","763,980.75910",0.00007
65,d_O_from1st_0,"542,893.52662","407,577.05864","522,879.46225","709,298.34846","801,393.86190","596,808.45157",0.00005
67,mean_dist_C_from1st,"570,900.98960","577,426.49378","561,287.75217","666,204.12254","450,796.57512","565,323.18664",0.00005


In [28]:
importance_df_y_high.feature.tolist()


['mean_dist_C_from2nd',
 'mean_angle_C_from2nd',
 'd_O_from2nd_ratio_0',
 'mean_dist_ratio_C_from2nd',
 'mean_dist_ratio_O_from1st',
 'mean_dist_ratio_O_from2nd',
 'd_O_from1st_ratio_0',
 'mean_angle_O_from2nd',
 'd_O_from1st_0',
 'mean_dist_C_from1st']